# Sizing: Proof of Concept

Edgar Hernandez

Overview: Develop an initial sizing of an aircraft with mission two parameters and estiamte score for mission 2

Once proof of concept is established, we can turn this into an object from some parameters and have the object simulate against a mission two simulator

Once the objects and simulation work, we can run multiple objects with varying parameters

## Sizing Process
1. Start with some input parameters: 
    - Aspect Ratio
    - Cruising speed
    - Ducks and pucks
2. Initialize the weight of the aircraft with some guess
3. With wing size, estimate weight buildup of all other components
    - Size motor continuous power from take-off
    - Size battery energy from energy needed to complete mission
        - Ensure that usable energy is less than 0.7 of total battery energy
    - Size tail
        - First cut use tail_weight/wing_area estimation
        - Later on the tail can be sized to provide stability, then weight can be estimated from tail_area
    - Size fuselage from ducks, pucks, battery, motor, and other electronics, esatimate drag build up of fuselage
        - First cut, omit other electronics and apply some safety factor, 
        - First cut, use battery and motor weight to estimate volume using a density (power/weight)
        - Second cut, improve models and estimate other electronics as well, perhaps from heritage planes
4. Calculate power needed at take off and cruise to complete mission
5. Size motor and battery, from power needed, and reiterate the performance of the aircraft to complete mission
6. Repeat until converged, error ~ 0
7. Document wing_loading and power_loading of aircraft, and estimate how it performs in mission two, i.e. mission two score


In [28]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def density_from_alt(height_asl):
    """ calculate density from altitude """
    theta = 1 + (-0.000022558) * height_asl
    rho = 1.225 * (theta**4.2561)
    return rho

def reynolds_number(freestream_speed, length_wet, rho):
    """ calculate the reynolds number in air from freestream """
    mu = 18*10**(-6) # Pa*s, need to get a better approximation for this as well
    re = rho * freestream_speed * length_wet / mu
    return re

def calculate_fuselage_drag(reynolds_number, fineness_ratio, S_wet, S_ref):
    """ calculate profile drag of fuselage using hoerner estimation for streamline shapes """
    cf_fuselage = 1.328 / np.sqrt(reynolds_number)
    CD_fuselage_wet = cf_fuselage*(1+(1/fineness_ratio)**(3/2)) + 0.11*(1/fineness_ratio)**2
    CD_fuselage= CD_fuselage_wet * S_wet / S_ref
    return CD_fuselage

def battery_mass_from_energy(battery_energy):
    """ estimate mass of battery from total energy capacitance 
    This can be done two ways
    1. energy density constant
    2. energy/mass density where it increases as energy increase
    """
    energy_density = 0.01952 # kg/Whr
    return (battery_energy*energy_density)

def battery_volume_from_mass(battery_mass):
    """ Calculate battery volume from mass """
    battery_mass_density = 1988.915 # kg/m^3
    return(battery_mass / battery_mass_density)

def motor_mass_from_power(max_continuous_shaft_power):
    """ estimate motor mass from power required to drive shaft/propeller at take off
    assuming linear increase in power and motor mass, could use a better model (FRESHMAN!!!!!) 
     """
    motor_power_loading = 5.6   # W/g
    return (max_continuous_shaft_power / motor_power_loading / 1000 )

def motor_volume_from_mass(motor_mass):
    """ Calculate battery mass from power"""
    motor_mass_density = 68.45  # kg/m^3
    return (motor_mass/motor_mass_density)

def calculate_take_off_power(wing_area, mass, CLmax, ground_run, CDtof, CLtof, rho, mu=0.05, gravity = 9.81):
    """ estimate power at battery to achieve take off, assuming a constant acceleration """
    wing_loading = mass*gravity/wing_area
    thrust = (1.21/gravity/rho/CLmax/ground_run*wing_loading + 0.605/CLmax*(CDtof-mu*CLtof) + mu)*mass*gravity
    available_power = thrust * np.sqrt(2*wing_loading/rho/CLmax)
    return available_power  # Watts, if everything is in SI

def calculate_time_to_tof(Vlof, ground_run):
    """ Calculate time to take off """
    acceleration = Vlof**2 / 2 / ground_run # m/s^2
    return (Vlof/acceleration)  # seconds

# If this were a class then I wouldn't need so many functions, but thats once this is done and working
def calculate_climb_rate(total_power, available_power, mass, gravity=9.81):
    excess_power = total_power - available_power
    return (excess_power/mass/gravity)  # climb rate, m/s?

def calculate_time_to_complete_climb(climb_rate, ceiling):
    """ calculate time to finish climb sequence """
    return (ceiling / climb_rate)

def calculate_power_needed_for_level_turn(mass, wing_area, airspeed, rho, bank_angle_rad, CDmin, k, gravity=9.81):
    """ calculate available power needed to complete a level turn at some angle and airspeed """
    wing_loading = mass*gravity/wing_area
    load_factor = 1/np.cos(bank_angle_rad)
    p_dynamic = 1/2*rho*airspeed**2
    thrust = mass*gravity*p_dynamic*(CDmin/wing_loading + k*((load_factor/p_dynamic)**2) * wing_loading)
    return(thrust*airspeed)

def calculate_power_needed_for_cruise(mass, wing_area, airspeed, rho, CDmin, k, gravity=9.81):
    """ calculate available power needed to cruise at steady state (ignore trim drag) """
    wing_loading = mass*gravity/wing_area
    p_dynamic = 1/2*rho*airspeed**2
    thrust = mass*gravity*(p_dynamic*CDmin/wing_loading + k/p_dynamic*wing_loading)
    return(thrust*airspeed)


# ----- Code inputs ---- #
# Wing parameters
AR = 5
b = 5 / 3.281
S = b**2 / AR
CLmax = 1
CDmin_no_fus = 0.03
# Mission two parameters 
cargo = 5
ducks = cargo * 3 + 20  
V_m2_cruise = 20   # m/s, use cruise speed instead of laps
phi = 45*np.pi/180  # rad, bank angle
time_limit = 60*5 / 1.1    # vary w/ some safety factor
# Flight course parameters
ground_run = 60 # m, need to get a more heuristic value for this
max_altitude = 200 / 3.281  # half of altitude limit
length_straight = 500 / 3.281

# ---- Initialize ---- #
# Atmosphere model
rho = density_from_alt(400)
# Simple drag model
k = 1 / (np.pi*AR*0.85)
# Initialize everything for sizing
sizing_error = 1
iteration = []
mass_m2_battery = []
mass_motor = []
mass_m2_gross = []
volume_m2_battery = []
volume_motor = []
volume_m2_electronics = []
volume_m2_gross = []
d = []
l = []
S_fuselage = []
V_m2_st = []
V_m2_tof = []
V_m2_v = []
Re_fus_tof = []
CD_fus_tof = []
Re_fus_cruise = []
CD_fus_cruise = []
CL_m2_tof = []
CL_m2_cruise = []
CD_m2_min_tof = []
CD_m2_min_cruise = []
CD_m2_tof = []
CD_m2_cruise = []
P_m2_tof_avail = [] # T*V at take off roll
P_m2_total_climb = []   # Total power at battery during climb 
P_m2_avail_climb = []   # Power at battery going to thrust during climb
P_m2_exc_climb = []     # Excess power at battery during climb, directly going to climb rate
P_m2_motor = []    # motor sized to take off power
P_m2_battery = []   # Power needed in battery sized to take off
time_tof = []
time_climb = []
time_cruise = []
time_turn_ea = []
time_straights_ea = []
time_circle_ea = []
time_m2_total = []
E_m2_tof = []
E_m2_climb = []
E_m2_cruise = []
E_m2_battery = []
L_m2_turn = []
radius_m2_turn = []
circuit_ground_length = []
laps_m2 = []
sizing_error_batt = []
sizing_error_motor = []


# Systems parameters, more or less placeholders for the time being, estimating power flow in take off and cruise
eff_propeller_tof = 0.4
eff_motor_tof = 0.9
eff_esc_tof = 0.9
eff_battery_tof = 0.95
eff_system_tof = eff_propeller_tof * eff_motor_tof * eff_esc_tof * eff_battery_tof
eff_propeller_cruise = 0.5
eff_motor_cruise = 0.9
eff_esc_cruise = 0.9
eff_battery_cruise = 0.95
eff_system_cruise = eff_propeller_cruise * eff_motor_cruise * eff_esc_cruise * eff_battery_cruise

# First cut weight estimation
mass_landing_gear = 0.500    # From UCLA landing gear, kg
mass_ducks =  ducks * 0.7 / 35.274    # mass of ducks, kg
mass_cargo = cargo * 6 / 35.274  # mass of cargo, kg
mass_wing = S / 3  # kg, replace w/ wing weight to wingspan ratio or something similar
mass_tail = mass_wing / 4   # kg

for i in range(100):
    iteration.append(i)
    if i == 0:
        """ initial guess """
        mass_m2_battery.append(0.080)  # Initial battery mass guess, kg
        mass_motor.append(0.05)  # Initial motor mass guess, kg
    else:
        """ refined mass build up """
        mass_m2_battery.append(battery_mass_from_energy(E_m2_battery[i-1]))
        mass_motor.append(motor_mass_from_power(P_m2_motor[i-1]))

    mass_m2_gross.append(mass_m2_battery[i] + mass_motor[i] + mass_landing_gear + mass_ducks + mass_cargo + mass_wing)  # kg

    # Estimate stalling speed and take off speed from mass
    V_m2_st.append(np.sqrt(2*mass_m2_gross[i]*9.81/rho/S/CLmax))
    V_m2_tof.append(1.1*V_m2_st[i])

    # ----- Fuselage Sizing ----- #
    volume_m2_battery.append(battery_volume_from_mass(mass_m2_battery[i]))
    volume_motor.append(motor_volume_from_mass(mass_motor[i]))
    volume_m2_electronics.append((volume_m2_battery[i] + volume_motor[i])/1.2)
    volume_m2_payload = (ducks*(2.5/39.37)**3 + cargo * np.pi/4/39.37)/1.2 # m^3
    volume_m2_gross.append(volume_m2_electronics[i]+volume_m2_payload)
    fineness_ratio = 6  # L/D
    d.append((volume_m2_gross[i]/fineness_ratio)**(1/3)) # m
    l.append(fineness_ratio * d[i]) # m
        # make sure to eventually include a check to make sure that d isn't too small and l isn't too big
    S_fuselage.append(4*d[i]*l[i] + 2*d[i]**2) #m^2
    # take-off drag
    Re_fus_tof.append(reynolds_number(V_m2_tof[i], l[i], rho))
    CD_fus_tof.append(calculate_fuselage_drag(Re_fus_tof[i], fineness_ratio, S_fuselage[i], S))
    # cruise drag
    Re_fus_cruise.append(reynolds_number(V_m2_cruise, l[i], rho))  #m^2/s^2/Pa
    CD_fus_cruise.append(calculate_fuselage_drag(Re_fus_cruise[i], fineness_ratio, S_fuselage[i], S))

    # ---- Estimate aerodynamics ----- #
    CL_m2_tof.append(mass_m2_gross[i]*9.81*2/rho/V_m2_tof[i]**2/S)
    CD_m2_min_tof.append(CDmin_no_fus + CD_fus_tof[i])
    CD_m2_tof.append(CD_m2_min_tof[i] + k*CL_m2_tof[i]**2)
    CL_m2_cruise.append(mass_m2_gross[i]*9.81*2/rho/V_m2_cruise**2/S)
    CD_m2_min_cruise.append(CDmin_no_fus + CD_fus_cruise[i])
    CD_m2_cruise.append(CD_m2_min_cruise[i] + k*CL_m2_cruise[i]**2)

    # ------ Estimate performance ----- #
    # Take off
    P_m2_tof_avail.append(calculate_take_off_power(S, mass_m2_gross[i], CLmax, ground_run, CD_m2_tof[i], CL_m2_tof[i], rho, mu=0.2))
    time_tof.append(calculate_time_to_tof(V_m2_tof[i], ground_run))
    E_m2_tof.append(P_m2_tof_avail[i]/eff_system_tof*time_tof[i]/3600)
    # Climb
    P_m2_total_climb.append(P_m2_tof_avail[i]/eff_system_tof)
    P_m2_avail_climb.append(CD_m2_tof[i]/2*rho*V_m2_tof[i]**3*S/eff_system_tof)
    P_m2_exc_climb.append(P_m2_total_climb[i]-P_m2_avail_climb[i])  # Not critical, but nice to store
    V_m2_v.append(calculate_climb_rate(P_m2_total_climb[i], P_m2_avail_climb[i], mass_m2_gross[i])) # m/s
    time_climb.append(calculate_time_to_complete_climb(V_m2_v[i], max_altitude))
    E_m2_climb.append(P_m2_total_climb[i]*time_climb[i]/36000)
    S_ground_climb_temp = V_m2_tof[i]*time_climb[i]     # need to adjust code to have roc be fixed and power sized to roc necessary
    # Cruise, pick up from where climb was completed, starting with first turn
    time_cruise_temp = time_limit - time_climb[i] - time_tof[i] # time limit after take off and climb
        # Turns
    L_m2_turn.append(mass_m2_gross[i]*9.81/np.cos(phi)) # total lift during turn
    radius_m2_turn.append(mass_m2_gross[i]*V_m2_cruise**2/L_m2_turn[i]/np.sin(phi))
    time_turn_ea.append(radius_m2_turn[i]/V_m2_cruise*(180*np.pi/180))
    time_circle_ea.append(time_turn_ea[i]*2)
    P_m2_turn_temp = calculate_power_needed_for_level_turn(mass_m2_gross[i], S, V_m2_cruise, rho, phi, CD_m2_min_cruise[i], k)/eff_system_cruise
        # Entire circuit
    time_straights_ea.append(304.8/V_m2_cruise)    # same for each unique cruising speed
    time_lap_1_temp = time_tof[i] + time_climb[i] + 2*time_turn_ea[i] + time_straights_ea[i] + time_circle_ea[i]
    time_after_lap_1_temp = time_cruise_temp - time_lap_1_temp  # How much time left after first lap in seconds
    time_ea_lap_temp = 2*time_straights_ea[i] + 2*time_turn_ea[i] + time_circle_ea[i]
    circuit_ground_length.append(2*304.8+4*radius_m2_turn[i])
    laps_m2.append(np.floor(1+time_after_lap_1_temp/time_ea_lap_temp))
    time_cruise.append(time_cruise_temp)
    n_turns_temp = laps_m2[i]*4
    n_straights_temp = laps_m2[i]*2 - 1
    time_m2_total.append(time_after_lap_1_temp + time_ea_lap_temp*(laps_m2[i]-1))
    P_m2_straights_temp = calculate_power_needed_for_cruise(mass_m2_gross[i], S, V_m2_cruise, rho, CD_m2_min_cruise[i], k)/eff_system_cruise
    E_m2_cruise.append((P_m2_turn_temp*n_turns_temp*time_turn_ea[i] + P_m2_straights_temp*n_straights_temp*time_straights_ea[i])/3600)

    # ----- Propulsion Sizing ------ #
    # Motor
    P_m2_motor.append(P_m2_tof_avail[i]/eff_propeller_tof)
    mass_motor_temp = motor_mass_from_power(P_m2_motor[i])
    error_motor = abs(mass_motor_temp-mass_motor[i])
    # Battery
    E_m2_battery.append(1/0.75*(E_m2_tof[i]+E_m2_climb[i]+E_m2_cruise[i]))
    mass_battery_temp = battery_mass_from_energy(E_m2_battery[i])
    error_battery = abs(mass_battery_temp-mass_m2_battery[i])
    # P_m2_battery.append()
    sizing_error_batt.append(error_battery)
    sizing_error_motor.append(error_motor)
    sizing_error = max(error_battery, error_motor)

    if sizing_error < 0.001:
        break

